In [1]:
import torch
import torch.nn as nn
from build_dataloader import get_train_loader, get_test_loader
from custom_resnet import CustomRes
from torchsummary import summary
from trainer import train, test
from torch.optim import SGD,Adam
from torch.optim.lr_scheduler import OneCycleLR
from ignite.handlers import FastaiLRFinder

/Users/raginipandey/anaconda3/envs/myenv/lib/python3.9/site-packages/albumentations/augmentations/dropout/cutout.py:49: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


In [2]:
BATCHSIZE=512
train_dataloader = get_train_loader(data_dir='./data', 
                                    train=True,
                                    download=True,
                                    shuffle=True,
                                    batch_size=BATCHSIZE
                                    )

test_dataloader = get_test_loader(data_dir='./data', 
                                    train=False,
                                    download=True,
                                    shuffle=True,
                                    batch_size=BATCHSIZE
                                    )

Files already downloaded and verified
Files already downloaded and verified


In [3]:
model = CustomRes()
summary(model, input_size=(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 32, 32]          73,728
         MaxPool2d-5          [-1, 128, 16, 16]               0
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,456
       BatchNorm2d-9          [-1, 128, 16, 16]             256
             ReLU-10          [-1, 128, 16, 16]               0
           Conv2d-11          [-1, 128, 16, 16]         147,456
      BatchNorm2d-12          [-1, 128, 16, 16]             256
             ReLU-13          [-1, 128, 16, 16]               0
           Conv2d-14          [-1, 256,

In [4]:
EPOCHS = 25
criterion = nn.CrossEntropyLoss()
optimizer = Adam(params=model.parameters(), lr=0.08)
scheduler = OneCycleLR(optimizer=optimizer, max_lr=.01, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device("mps")
for e in range(EPOCHS):
    train(model, optimizer=optimizer, train_dataloader=train_dataloader,device=device,epoch=e,scheduler=scheduler, criterion=criterion )
    test(model,test_dataloader=test_dataloader, device=device, epoch=e,criterion=criterion)

  0%|          | 0/98 [00:00<?, ?it/s]

100%|██████████| 98/98 [00:35<00:00,  2.76it/s]


at epoch:0
avg_train_loss:1.416864037513733,avg_train_acc:0.5007924437522888
avg_test_loss:1.0424970388412476,avg_test_acc:0.6290326118469238


 16%|█▋        | 16/98 [00:05<00:29,  2.77it/s]